# Optimal Control Problem

We tackle the stabilization of an inverted pendulum via the ***Optimal Energy Shaping*** approach of [CIT]

In [ ]:
import torch
from torch.distributions import Uniform, Normal

def prior_dist(q_min, q_max, p_min, p_max, device='cpu'):
    # uniform "prior" distribution of initial conditions x(0)=[q(0),p(0)]
    lb = torch.Tensor([q_min, p_min]).to(device)
    ub = torch.Tensor([q_max, p_max]).to(device)
    return Uniform(lb, ub)

def target_dist(mu, sigma, device='cpu'):
    # normal target distribution of terminal states x(T)
    mu, sigma = torch.Tensor(mu).reshape(1, 2).to(device), torch.Tensor(sigma).reshape(1, 2).to(device)
    return Normal(mu, torch.sqrt(sigma))

def weighted_log_likelihood_loss(x, target, weight):
    # weighted negative log likelihood loss
    log_prob = target.log_prob(x)
    weighted_log_p = weight * log_prob
    return -torch.mean(weighted_log_p.sum(1))

In [ ]:
def dummy_trainloader():
    # dummy trainloader for Lightning learner. We will sample new initial conditions at each training iteration
    dummy = data.DataLoader(
        data.TensorDataset(
            torch.Tensor(1, 1),
            torch.Tensor(1, 1)
        ),
        batch_size=1,
        shuffle=False
    )
    return dummy

## Define Controlled System Dynamics

We consider a controlled damped inverted pendulum with a torsional spring in the joint with state $(q_t, p_t)$ (joint angle and momentum), described by the Hamiltonian dynamics
$$
    \begin{aligned}
        \dot q_t &= \frac{1}{m}p_t \\
        \dot p_t &= \tau_{el} + \tau_{g} + \tau_{d} + u_t
    \end{aligned}
$$

with input torque $u_t$, elastic torque $\tau_{el} = - k (q - q_r)$, gravitational torque $\tau_g = mgl\sin q_t$ and damping (friction) $\tau_d = \frac{b}{m} p_t$.

Following [CITE] the controller is obtained as 

$$
    u_t = \gamma_\theta(q_t, p_t) = -\nabla_q V_\theta^*(q_t) - \frac{1}{m}K(q_t, p_t)p_t
$$

with $V^*_\theta:\mathbb R \rightarrow\mathbb R, ~K^*_\theta:\mathbb R^2 \rightarrow\mathbb R$ being two neural nets with parameters $\theta$


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import grad as grad

# physical parameters
m, k, l, qr, b, g = 1., 0.5, 1, 0, 0.01, 9.81

class ControlledSystem(nn.Module):
    # Elastic Pendulum Model
    def __init__(self, V, K):
        super().__init__()
        self.V, self.K, self.n = V, K, 1

    def forward(self, t, x):
        # Evaluates the closed-loop vector field
        with torch.set_grad_enabled(True):
            q, p = x[..., :self.n], x[..., self.n:]
            q = q.requires_grad_(True)
            # compute control action
            u = self._energy_shaping(q) + self._damping_injection(x)
            # compute dynamics
            dxdt = self._dynamics(q, p, u)
        return dxdt

    def _dynamics(self, q, p, u):
        # controlled elastic pendulum dynamics
        dqdt = p / m
        dpdt = -k * (q - qr) - m * g * l * torch.sin(q) - b * p / m + u
        return torch.cat([dqdt, dpdt], 1)

    def _energy_shaping(self, q):
        # energy shaping control action
        dVdx = grad(self.V(q).sum(), q, create_graph=True)[0]
        return -dVdx

    def _damping_injection(self, x):
        # damping injection control action
        return -self.K(x) * x[:, self.n:] / m

    def _autonomous_energy(self, x):
        # Hamiltonian (total energy) of the UNCONTROLLED system
        return (m * x[:, 1:] ** 2) / 2. + (k * (x[:, :1] - qr) ** 2) / 2 \
               + m * g * l * (1 - torch.cos(x[:, :1]))

    def _energy(self, x):
        # Hamiltonian (total energy) of the CONTROLLED system
        return (m * x[:, 1:] ** 2) / 2. + (k * (x[:, :1] - qr) ** 2) / 2 \
               + m * g * l * (1 - torch.cos(x[:, :1])) + self.V(x[:, :1])


class AugmentedDynamics(nn.Module):
    # "augmented" vector field to take into account integral loss functions
    def __init__(self, f, int_loss):
        super().__init__()
        self.f = f
        self.int_loss = int_loss
        self.nfe = 0.

    def forward(self, t, x):
        self.nfe += 1
        x = x[:,:2]
        dxdt = self.f(t, x)
        dldt = self.int_loss(t, x)
        return torch.cat([dxdt, dldt], 1)

In [3]:
from sys import path ; path.append('../../')
from torchdyn.numerics import odeint




from src.control.utils import prior_dist, target_dist, dummy_trainloader, weighted_log_likelihood_loss
from src.control.models import ControlledSystem, AugmentedDynamics
from src.control.learners import EnergyShapingLearner

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'src'